In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
from feature_manager import FeatureManager
import dnn_multi_class_classifier as dnn
from random import randint
from keras import callbacks
from keras import losses
from visualizer import *
from keras.utils import np_utils
from sklearn.metrics import classification_report
import importlib


2023-01-17 23:31:37.662482: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-17 23:31:37.662573: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-17 23:31:37.662583: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:
timeframe_in_ms = {"15m":15*60*1000,"1h":60*60*1000,"1d":24*60*60*1000}

symbol = "BTCUSDT"
trade_tf = "1h"
granular_tf = "1m"

tp = 0.05
sl = 0.025
md = 7


data = pd.read_csv("../data/{}-{}.csv".format(
    symbol,trade_tf), 
    parse_dates=["Open Time"], 
    index_col = "Open Time"
)

granular_data = pd.read_csv("../nocommit/{}-{}.csv".format(
    symbol,granular_tf), 
    parse_dates=["Open Time"], 
    index_col = "Open Time"
)

fm: FeatureManager = FeatureManager()
fm.build_feature(data=data,lags=5)

fm.prepare_trade_forward_data(
    data=fm.df,
    take_profit_rate=tp,
    stop_loss_rate=sl,
    max_duration=md,
    granular_data=granular_data,
    timeframe_in_ms=timeframe_in_ms[trade_tf]
)
# loop_classifier(data=fm.df,cols=fm.cols,target_col="trade_signal",laps=3)

Processing 1/7
Processing 2/7
Processing 3/7
Processing 4/7
Processing 5/7
Processing 6/7
Processing 7/7


In [3]:
fm.df["trade_signal"].value_counts()

0    42621
2     2368
1     1839
Name: trade_signal, dtype: int64

In [6]:
classifier = dnn.MultiClassDNNClassifer()
classifier.prepare_data(data = fm.df,cols = fm.cols,target_col="trade_signal",rebalance="under",y_to_categorical = True, random_state=randint(0,100))

In [7]:
classifier.configure(hu=100,input_dim=len(fm.cols),class_num=3)
classifier.run()

Epoch 1/200

Epoch 1: val_loss improved from inf to 1.03935, saving model to ../data/model_dnn_checkpoint.h5
386/386 - 2s - loss: 1.0875 - accuracy: 0.4283 - val_loss: 1.0393 - val_accuracy: 0.4551 - 2s/epoch - 5ms/step
Epoch 2/200

Epoch 2: val_loss improved from 1.03935 to 1.01173, saving model to ../data/model_dnn_checkpoint.h5
386/386 - 1s - loss: 0.9897 - accuracy: 0.4987 - val_loss: 1.0117 - val_accuracy: 0.4705 - 1s/epoch - 3ms/step
Epoch 3/200

Epoch 3: val_loss improved from 1.01173 to 1.00112, saving model to ../data/model_dnn_checkpoint.h5
386/386 - 1s - loss: 0.9594 - accuracy: 0.5260 - val_loss: 1.0011 - val_accuracy: 0.5038 - 1s/epoch - 3ms/step
Epoch 4/200

Epoch 4: val_loss improved from 1.00112 to 0.99532, saving model to ../data/model_dnn_checkpoint.h5
386/386 - 1s - loss: 0.9401 - accuracy: 0.5376 - val_loss: 0.9953 - val_accuracy: 0.5244 - 1s/epoch - 3ms/step
Epoch 5/200

Epoch 5: val_loss improved from 0.99532 to 0.98321, saving model to ../data/model_dnn_checkpoin

In [ ]:
importlib.reload(dnn)

In [ ]:
classifier = dnn.MultiClassDNNClassifer()
classifier.prepare_data(data = fm.df,cols = fm.cols,target_col="trade_signal",y_to_categorical=True, rebalance="under",random_state=randint(0,100))
classifier.configure(hu=100,input_dim=len(fm.cols),class_num=3)
classifier.run()